In [29]:
import torch
import torch.serialization
import pickle
import pandas as pd

In [17]:
# CUDA 가용성 확인
cuda_available = torch.cuda.is_available()
if cuda_available:
    print(f"CUDA 사용 가능: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
else:
    print("CUDA를 사용할 수 없어 CPU를 사용합니다.")
    device = torch.device("cpu")

CUDA 사용 가능: NVIDIA GeForce RTX 4080 SUPER


In [30]:
# Path 설정
model_path = '../result/bmed_model.pt'
weights_path = '../result/bmed_model_weights.pt'
scalers_path = '../result/bmed_model_scalers.pkl'
data_path = '../data/BMED_data_v6+spline.xlsx'


In [19]:
# 모델 클래스 임포트
from BMED_LSTM_v4 import MembraneSystemModel, PhysicsLayer
# 불러온 클래스를 safe globals에 등록
import torch.serialization
torch.serialization.add_safe_globals([MembraneSystemModel, PhysicsLayer])




In [24]:
# 모델 로드
model = torch.load(model_path, map_location=device, weights_only=False)
# 모델 GPU 확인
next_param = next(model.parameters(), None)
if next_param is not None:
    print(f"모델 파라미터 디바이스: {next_param.device}")

모델 파라미터 디바이스: cuda:0


In [28]:
# 모델 평가모드 설정
model.eval()
# 스케일러 로드
with open(scalers_path, 'rb') as f:
    scalers = pickle.load(f)

In [35]:
# 데이터 로드
df_raw = pd.read_excel(data_path,sheet_name='raw_data')
# exp 1 추출
exp1_data = df_raw[df_raw['exp'] == 1].sort_values('t')
# 초기값 추출
cond_init = exp1_data.iloc[0]
cond_init

exp       1.000000
T        24.816667
V        20.000000
E         0.250000
Ci        0.500000
Ki        1.000000
t         0.000000
CF_LA     0.500000
CA_LA     0.000000
CF_K      1.000000
CB_K      0.000000
VF        1.000000
VA        1.000000
VB        1.000000
Name: 0, dtype: float64